In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder().appName("sesion_1").master("local[*]").getOrCreate()
val sc = spark.sparkContext

sc = org.apache.spark.SparkContext@2aedbf9d


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@27825b45


org.apache.spark.SparkContext@2aedbf9d

In [2]:
sc.uiWebUrl

Some(http://23LAP5CD0176GH7:4041)

In [3]:
val contractsDf  = spark.read.option("header","true")
        .option("delimiter",",")
        .option("inferSchema","true")
        .csv("../../resources/data/csv/contracts.csv")

contractsDf = [cod_iuc: int, cod_titular: int ... 3 more fields]


[cod_iuc: int, cod_titular: int ... 3 more fields]

In [4]:
contractsDf.show(2)

+-------+-----------+------------+----------+------+
|cod_iuc|cod_titular|cod_producto|  fec_alta|activo|
+-------+-----------+------------+----------+------+
|  30000|          6|         100|2012-05-01|  true|
|  30001|          6|         200|2014-05-01|  true|
+-------+-----------+------------+----------+------+
only showing top 2 rows



In [5]:
contractsDf.printSchema()

root
 |-- cod_iuc: integer (nullable = true)
 |-- cod_titular: integer (nullable = true)
 |-- cod_producto: integer (nullable = true)
 |-- fec_alta: date (nullable = true)
 |-- activo: boolean (nullable = true)



#### - Clase Column
#### - Package Types
#### - Object functions
#### - Clase Dataframe

### Transformaciones

In [22]:
import org.apache.spark.sql.{functions => f}
import org.apache.spark.sql.{types => t}

val resultedDf = contractsDf
    .withColumnRenamed("fec_alta", "fec_alta_ini")
    .withColumn("actual_date", f.current_date())
    .withColumn("randm_num", f.round(f.rand(0) * f.lit(10)).cast(t.IntegerType))
    .withColumn("fec_alta_fin", f.date_add(f.col("fec_alta_ini"), f.col("randm_num")))
    .withColumn("diff", f.datediff(f.col("fec_alta_fin"), f.col("fec_alta_ini")))
    .drop("randm_num")

resultedDf = [cod_iuc: int, cod_titular: int ... 6 more fields]


import org.apache.spark.sql.{functions=>f}
import org.apache.spark.sql.{types=>t}


[cod_iuc: int, cod_titular: int ... 6 more fields]

### Acciones

In [7]:
resultedDf.show(2, false)

+-------+-----------+------------+------------+------+-----------+------------+----+
|cod_iuc|cod_titular|cod_producto|fec_alta_ini|activo|actual_date|fec_alta_fin|diff|
+-------+-----------+------------+------------+------+-----------+------------+----+
|  30000|          6|         100|  2012-05-01|  true| 2024-01-16|  2012-05-09|   8|
|  30001|          6|         200|  2014-05-01|  true| 2024-01-16|  2014-05-06|   5|
+-------+-----------+------------+------------+------+-----------+------------+----+
only showing top 2 rows



In [8]:
resultedDf.count()

32

In [9]:
resultedDf.collect() // WARNING

Array([30000,6,100,2012-05-01,true,2024-01-16,2012-05-09,8], [30001,6,200,2014-05-01,true,2024-01-16,2014-05-06,5], [30002,6,300,2006-02-01,false,2024-01-16,2006-02-02,1], [30003,6,150,2012-05-01,true,2024-01-16,2012-05-04,3], [30002,5,300,2012-05-01,true,2024-01-16,2012-05-08,7], [30004,6,400,2012-05-01,false,2024-01-16,2012-05-04,3], [30005,6,500,2012-05-01,true,2024-01-16,2012-05-03,2], [30006,6,600,2012-05-01,false,2024-01-16,2012-05-07,6], [30003,3,150,2019-10-14,true,2024-01-16,2019-10-21,7], [30007,6,700,2014-02-01,false,2024-01-16,2014-02-01,0], [30008,6,800,2012-05-01,true,2024-01-16,2012-05-03,2], [30009,6,900,2015-09-01,true,2024-01-16,2015-09-07,6], [30009,2,900,2009-10-01,true,2024-01-16,2009-10-05,4], [30010,6,1000,2014-02-01,true,20...

In [25]:
resultedDf.isEmpty

false

In [11]:
//first() -> primer elemento del df -> head()
//head(n) -> primer N-elementos del df -> take(n)
//take -> primer N-elementos del df -> limit().collect()

resultedDf.take(4)

Array([30000,6,100,2012-05-01,true,2024-01-16,2012-05-09,8], [30001,6,200,2014-05-01,true,2024-01-16,2014-05-06,5], [30002,6,300,2006-02-01,false,2024-01-16,2006-02-02,1], [30003,6,150,2012-05-01,true,2024-01-16,2012-05-04,3])

In [12]:
resultedDf.tail(4)

Array([30008,7,800,2004-09-01,true,2024-01-16,2004-09-10,9], [30008,2,800,2014-03-21,true,2024-01-16,2014-03-29,8], [30004,2,400,2008-02-01,false,2024-01-16,2008-02-10,9], [30010,7,1000,2014-08-01,false,2024-01-16,2014-08-08,7])

In [26]:
resultedDf.describe("cod_iuc","cod_titular").show()

+-------+------------------+------------------+-----------------+------------------+
|summary|           cod_iuc|       cod_titular|     cod_producto|              diff|
+-------+------------------+------------------+-----------------+------------------+
|  count|                32|                32|               32|                32|
|   mean|        30005.0625|            4.4375|         504.6875|           4.65625|
| stddev|3.1819805153393363|2.0150642349482912|310.1402843355599|2.8239685527811345|
|    min|             30000|                 1|              100|                 0|
|    max|             30011|                 7|             1100|                 9|
+-------+------------------+------------------+-----------------+------------------+



In [27]:
resultedDf.summary("count", "stddev").show()

+-------+------------------+------------------+-----------------+------------------+
|summary|           cod_iuc|       cod_titular|     cod_producto|              diff|
+-------+------------------+------------------+-----------------+------------------+
|  count|                32|                32|               32|                32|
| stddev|3.1819805153393363|2.0150642349482912|310.1402843355599|2.8239685527811345|
+-------+------------------+------------------+-----------------+------------------+



### Functions

In [15]:
resultedDf.columns

Array(cod_iuc, cod_titular, cod_producto, fec_alta_ini, activo, actual_date, fec_alta_fin, diff)

In [31]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.Row

val schema_nest = StructType( Array(
    StructField("alumno",StructType( Array(
      StructField("nombre", StringType,true),
      StructField("apellido_p", StringType,true),
      StructField("apellido_m", StringType,true)
    ))),
    StructField("calificaciones", StructType( Array(
        StructField("matematicas",StructType( Array(
            StructField("periodo_1", IntegerType,true),
            StructField("periodo_2", IntegerType,true),
            StructField("periodo_3", IntegerType,true)
        ))),
        StructField("espanol",StructType( Array(
            StructField("periodo_1", IntegerType,true),
            StructField("periodo_2", IntegerType,true),
            StructField("periodo_3", IntegerType,true)
        ))),
    )))
  ))

//Create DataFrame
val data = List(
    Row(Row("Pedro","Jimenez","Perez"), Row(Row(10, 9, 8), Row(8, 9, 10))),
    Row(Row("Juan","Hernandez","Perez"), Row(Row(10, 10, 8), Row(9, 9, 10)))
    )
val df = spark.createDataFrame(
    spark.sparkContext.parallelize(data),schema_nest)
df.show(2, false)
df.printSchema()

schema_nest: org.apache.spark.sql.types.StructType = StructType(StructField(alumno,StructType(StructField(nombre,StringType,true),StructField(apellido_p,StringType,true),StructField(apellido_m,StringType,true)),true),StructField(calificaciones,StructType(StructField(matematicas,StructType(StructField(periodo_1,IntegerType,true),StructField(periodo_2,IntegerType,true),StructField(periodo_3,IntegerType,true)),true),StructField(espanol,StructType(StructField(periodo_1,IntegerType,true),StructField(periodo_2,IntegerType,true),StructField(periodo_3,IntegerType,true)),true)),true))
data: List[org.apache.spark.sql.Row] = List([[Pedro,Jimenez,Perez],[[10,9,8],[8,9,10]]], [[Juan,Hernandez,Perez],[[10,10,8],[9,9,10]]])
df: or...


+------------------------+-------------------------+
|alumno                  |calificaciones           |
+------------------------+-------------------------+
|{Pedro, Jimenez, Perez} |{{10, 9, 8}, {8, 9, 10}} |
|{Juan, Hernandez, Perez}|{{10, 10, 8}, {9, 9, 10}}|
+------------------------+-------------------------+

root
 |-- alumno: struct (nullable = true)
 |    |-- nombre: string (nullable = true)
 |    |-- apellido_p: string (nullable = true)
 |    |-- apellido_m: string (nullable = true)
 |-- calificaciones: struct (nullable = true)
 |    |-- matematicas: struct (nullable = true)
 |    |    |-- periodo_1: integer (nullable = true)
 |    |    |-- periodo_2: integer (nullable = true)
 |    |    |-- periodo_3: integer (nullable = true)
 |    |-- espanol: struct (nullable = true)
 |    |    |-- periodo_1: integer (nullable = true)
 |    |    |-- periodo_2: integer (nullable = true)
 |    |    |-- periodo_3: integer (nullable = true)



In [34]:
df.printSchema(3)

root
 |-- alumno: struct (nullable = true)
 |    |-- nombre: string (nullable = true)
 |    |-- apellido_p: string (nullable = true)
 |    |-- apellido_m: string (nullable = true)
 |-- calificaciones: struct (nullable = true)
 |    |-- matematicas: struct (nullable = true)
 |    |    |-- periodo_1: integer (nullable = true)
 |    |    |-- periodo_2: integer (nullable = true)
 |    |    |-- periodo_3: integer (nullable = true)
 |    |-- espanol: struct (nullable = true)
 |    |    |-- periodo_1: integer (nullable = true)
 |    |    |-- periodo_2: integer (nullable = true)
 |    |    |-- periodo_3: integer (nullable = true)



In [17]:
resultedDf.schema

StructType(StructField(cod_iuc,IntegerType,true),StructField(cod_titular,IntegerType,true),StructField(cod_producto,IntegerType,true),StructField(fec_alta_ini,DateType,true),StructField(activo,BooleanType,true),StructField(actual_date,DateType,false),StructField(fec_alta_fin,DateType,true),StructField(diff,IntegerType,true))

In [18]:
resultedDf.dtypes

Array((cod_iuc,IntegerType), (cod_titular,IntegerType), (cod_producto,IntegerType), (fec_alta_ini,DateType), (activo,BooleanType), (actual_date,DateType), (fec_alta_fin,DateType), (diff,IntegerType))

In [19]:
resultedDf.rdd.collect().foreach(println)

[30000,6,100,2012-05-01,true,2024-01-16,2012-05-09,8]
[30001,6,200,2014-05-01,true,2024-01-16,2014-05-06,5]
[30002,6,300,2006-02-01,false,2024-01-16,2006-02-02,1]
[30003,6,150,2012-05-01,true,2024-01-16,2012-05-04,3]
[30002,5,300,2012-05-01,true,2024-01-16,2012-05-08,7]
[30004,6,400,2012-05-01,false,2024-01-16,2012-05-04,3]
[30005,6,500,2012-05-01,true,2024-01-16,2012-05-03,2]
[30006,6,600,2012-05-01,false,2024-01-16,2012-05-07,6]
[30003,3,150,2019-10-14,true,2024-01-16,2019-10-21,7]
[30007,6,700,2014-02-01,false,2024-01-16,2014-02-01,0]
[30008,6,800,2012-05-01,true,2024-01-16,2012-05-03,2]
[30009,6,900,2015-09-01,true,2024-01-16,2015-09-07,6]
[30009,2,900,2009-10-01,true,2024-01-16,2009-10-05,4]
[30010,6,1000,2014-02-01,true,2024-01-16,2014-02-06,5]
[30003,2,150,2018-09-18,true,2024-01-16,2018-09-22,4]
[30011,3,1100,2018-10-01,true,2024-01-16,2018-10-04,3]
[30007,6,700,2012-05-01,true,2024-01-16,2012-05-02,1]
[30007,2,700,2015-05-15,true,2024-01-16,2015-05-20,5]
[30002,7,300,2001-04-1

In [42]:
resultedDf.explain("formatted")

== Physical Plan ==
* Project (4)
+- * Project (3)
   +- * Project (2)
      +- Scan csv  (1)


(1) Scan csv 
Output [5]: [cod_iuc#17, cod_titular#18, cod_producto#19, fec_alta#20, activo#21]
Batched: false
Location: InMemoryFileIndex [file:/C:/Users/Administrador/Documents/curso_pyspark/spark_databricks_cert/resources/data/csv/contracts.csv]
ReadSchema: struct<cod_iuc:int,cod_titular:int,cod_producto:int,fec_alta:date,activo:boolean>

(2) Project [codegen id : 1]
Output [6]: [cod_iuc#17, cod_titular#18, cod_producto#19, fec_alta#20 AS fec_alta_ini#769, activo#21, cast(round((rand(0) * 10.0), 0) as int) AS randm_num#782]
Input [5]: [cod_iuc#17, cod_titular#18, cod_producto#19, fec_alta#20, activo#21]

(3) Project [codegen id : 1]
Output [6]: [cod_iuc#17, cod_titular#18, cod_producto#19, fec_alta_ini#769, activo#21, date_add(fec_alta_ini#769, randm_num#782) AS fec_alta_fin#790]
Input [6]: [cod_iuc#17, cod_titular#18, cod_producto#19, fec_alta_ini#769, activo#21, randm_num#782]

(4) Proj